In [32]:
require('nn')
require('hdf5')
require('optim')

In [33]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
n_reduced = 1001
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [34]:
function changeIndex(X, window_size, nfeatures)
    for i=1,X:size(2) do
        X:select(2,i):add((i-1)*nfeatures)
    end
    return X
end

function lookup(X, Y, win_size, C, K)
    W = torch.zeros(C, win_size*(K))
    for i=1,X:size(1) do
        for j=1,X:size(2) do
            local row = Y[i]
            local col = X[i][j]
            W[row][col] = W[row][col] + 1
        end
    end
    return W
end

In [45]:
function CBM(fc, Y, alpha, V)
    --calculate log posterior
    local cc = torch.zeros(1, V)
    local clp = torch.zeros(1, V)

    --collapse column wise so Nx1
    local hist = torch.histc(Y:double(), 10000)
    cc:add(torch.reshape(hist, 1, 10000))
    --THIS IS ALPHA
    --smoothing
    fc:add(alpha)
    local scc = fc:sum(2)
    
    fc:log()
    scc:log()
    
    scc:expand(scc, scc:size(1), fc:size(2))
    
    --calculate log prior
    --collapse down so 1x1
    local total = cc:sum(2)
    cc:div(total[1][1])
    cc:log()
    --clp:csub(math.log(Y:size(2)))
    
    return fc:csub(scc), cc
end

function predict(X, W, b)
  local jll = torch.zeros(X:size(1), W:size(1))
  for i = 1, X:size(1) do
    jll[i] = W:index(2, X[i]):sum(2)
    --right now prior is messed up
    jll[i]:add(b)
  end
  return jll
end

In [86]:
function perp(pred, tolookup)
    local temp = torch.zeros(tolookup:size(1), 50)
    for i=1, tolookup:size(1) do
        local total=0
        for j=1,tolookup:size(2) do
            idx = tolookup[i][j]
            temp[i][j] = pred[i][j] * -1
            total = -1 * pred[i][idx] + total
        end

    end
    return temp
end

In [37]:
X_train = changeIndex(X_train, 5, nfeatures)
X_valid = changeIndex(X_valid, 5, nfeatures)

In [38]:
W = lookup(X_train, Y_train, window_size, 10000, nfeatures)

In [46]:
W_, b = CBM(W, Y_train, 1, 10000)

In [48]:
jll = predict(X_valid, W_, b)

In [87]:
perp(jll, q_valid:sub(1,1))

Columns 1 to 6
  4.5307e+01   3.8242e+01          inf   4.6229e+01   4.1575e+01   4.9654e+01

Columns 7 to 12
  4.9626e+01   4.7587e+01   4.9427e+01   4.9471e+01   5.0504e+01   5.0029e+01

Columns 13 to 18
  4.8976e+01   4.5258e+01   5.2606e+01   5.0530e+01   5.2213e+01   4.9544e+01

Columns 19 to 24
  5.1646e+01   5.0465e+01   5.1395e+01   4.9982e+01   5.0912e+01   5.0924e+01

Columns 25 to 30
  5.0114e+01   5.3840e+01   5.6259e+01   5.5232e+01   5.0256e+01   5.2792e+01

Columns 31 to 36
  5.2654e+01   5.4149e+01   5.1882e+01   5.7145e+01   5.2582e+01   5.4567e+01

Columns 37 to 42
  4.5403e+01   5.2059e+01   4.4539e+01   5.3477e+01   5.2909e+01   4.9190e+01

Columns 43 to 48
  5.3973e+01   5.3762e+01   5.1295e+01   5.2707e+01   4.7738e+01   5.5473e+01

Columns 49 to 50
  4.6137e+01   5.2143e+01
[torch.DoubleTensor of size 1x50]



In [23]:
asdf = torch.Tensor{{1},{2},{2},{3},{2},{1}}

In [43]:
a = torch.histc(Y_train, 10000)

[string "a = torch.histc(Y_train, 10000)..."]:1: torch.LongTensor does not implement the torch.histc() function
stack traceback:
	[C]: in function 'histc'
	[string "a = torch.histc(Y_train, 10000)..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/Vincent/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0102f12bd0: 

In [44]:
Y_train:size()

 845453
[torch.LongStorage of size 1]

